In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import os
import re


In [3]:
driver = webdriver.Chrome(executable_path=r"D:\Tools\Chrome_Driver\chromedriver\chromedriver.exe")

In [4]:
url = "https://simple.wikipedia.org/wiki/List_of_countries"
driver.get(url)
print("URL:", driver.current_url)
time.sleep(2)

URL: https://simple.wikipedia.org/wiki/List_of_countries


In [5]:
root = driver.find_element(By.ID, "mw-content-text")

In [6]:
from selenium.webdriver.common.by import By

root = driver.find_element(By.ID, "mw-content-text")
a_elems = root.find_elements(By.TAG_NAME, "a")
len(a_elems)

362

In [7]:
import re

def is_country_link(a):
    txt = a.text.strip()
    href = a.get_attribute("href") or ""
    if not txt:
        return False
    if txt == "Image":
        return False
    if len(txt) > 40: 
        return False
    if "/wiki/" not in href:
        return False

    bad_ns = (":", "#") 
    if any(x in href.split("/wiki/")[-1] for x in bad_ns):
        return False
    if not re.match(r"^[A-Za-zÀ-ÿ]", txt):
        return False
    return True

country_links = [a for a in a_elems if is_country_link(a)]
len(country_links)

361

In [8]:
def clean_name(s):
    s = re.sub(r"\s+", " ", s).strip()
    s = re.sub(r"\[\d+\]", "", s)
    return s

countries = sorted(set(clean_name(a.text) for a in country_links))
print("Number of countires:", len(countries))
countries[:30]

Number of countires: 358


['Abkhazia',
 'Adjara',
 'Adygea',
 'Adélie Land',
 'Afghanistan',
 'Agaléga',
 'Akrotiri',
 'Albania',
 'Alderney',
 'Algeria',
 'Altai',
 'Alto Velo Island',
 'American Samoa',
 'Amsterdam Island',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Argentine Antarctica',
 'Armenia',
 'Artsakh',
 'Aruba',
 'Ascension Island',
 'Ashmore and Cartier Islands',
 'Australia',
 'Australian Antarctic Territory',
 'Austria',
 'Aves Island',
 'Azerbaijan']

In [9]:
out_path = os.path.join("..", "Data", "countries_lookup.csv")

pd.DataFrame({"country": countries}).to_csv(out_path, index=False, encoding="utf-8")